# Preparation

This part will help you get access to Gemini API and run your first prompting task.

For a detailed API walkthrough, see [this documentation page](https://ai.google.dev/tutorials/python_quickstart).


In [1]:
import sys
import os
import logging
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Setup your API key
Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in [Google AI Studio](https://makersuite.google.com/app/apikey). After obtaining the API key add it to your environment variables.

In [16]:
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## List models

Now you're ready to call the Gemini API. Use list_models to see the available Gemini models:


In [18]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


## Use this part to do your prompting

use the `gemini-pro` model.

In [10]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content("What is HCI?")
to_markdown(response.text)

> **HCI** stands for **Human-Computer Interaction**.
> 
> It is the study of how people interact with computers and other related devices. It encompasses a wide range of topics, including:
> 
> * **User experience (UX)**: The overall experience that users have when interacting with a computer system. This includes factors such as the system's ease of use, usability, and aesthetics.
> * **User interface (UI)**: The part of a computer system that users interact with directly. This includes the graphical user interface (GUI), command line interface (CLI), and other forms of interaction.
> * **Accessibility:** The ability of users with disabilities to interact with computer systems. This includes providing alternative forms of input and output, such as screen readers and closed captioning.
> * **Ergonomics:** The study of how computer systems can be designed to minimize discomfort and injury for users. This includes factors such as the design of the keyboard and mouse, and the positioning of the computer screen.
> 
> HCI is a multidisciplinary field that draws on knowledge from psychology, computer science, design, and other disciplines. It is an important field because it can help to make computer systems more usable, accessible, and enjoyable for users.

# Writeup 0: Task and Strategy Description

Here, you should describe what task you are working on, and what workflow/pipeline you intend to replicate (from which crowdsourcing paper). As a reminder, you should [pick a crowdsourcing paper here](https://docs.google.com/spreadsheets/d/1qM66oZ5YcSjqifegoyNgl3LFMaCfFHgoPe8isgckbXA/edit?usp=sharing). The spreadsheet also points to example tasks in the crowdsourcing papers; However, you DON'T have to stick to the paper-provided input/output. Please feel free to come up with your own tasks as long as they seem suitable for the paper/pipeline you are replicating.
 

**EDIT THIS PART TO PROVIDE AN OVERVIEW OF YOUR ATTEMPTS**

- **Task Description**: (description of your testing task.)
- **Example Input/output**: Write >=3 input-output pairs of your task. You should test your strategy on all the three examples.
    ```
    Input: [INPUT EXAMPLE]
    Output: [OUTPUT EXAMPLE]
    
    Input: [INPUT EXAMPLE]
    Output: [OUTPUT EXAMPLE]
    
    Input: [INPUT EXAMPLE]
    Output: [OUTPUT EXAMPLE]
    ```
- **Workflow prompting strategy**: Describe your designed pipeline.
- **Crowdsourcing paper**: [PAPER TITLE & URL](paper_url) -- where the pipelining strategy comes from.

## Baseline prompting

Use this section to do a one-step prompting for your selected task.

In [1]:
# Write your code here.

## Crowdsourcing-Pipeline-inspired prompting

Use this section to your selected pipeline prompting for your task.

In [2]:
# Write your code here.

# Writeup 1: Report & Reflection

Fill in the following three sections by reflecting on your results.

## Report the result

For all the inputs you tried, summarized the input, baseline output, pipeline output, which output you like and why. To answer "why", you should first think of some criteria you want to use for evaluating the output:

### Important criteria:
1. Diversity: The output should be diverse.
2. Clarity: The output should be descriptive and clearly explains why a metaphor is used.

### Inputs and outputs:

- **Input**: loss
- **Baseline output**: (diversity: 5/5, clarity: 1/5)
    - His heart was an empty void. 
    - The grief was a heavy weight on his shoulders. 
    - He felt like he was walking through a dark tunnel.

- **Pipeline output**: (diversity: 5/5, clarity: 1/5)
    - The world was a barren desert, devoid of hope and full of desolation.
    - Disappointment settled over the room like a thick fog, obscuring the joy that had been there before.
    - The grief of loss was like a balloon set free, slowly fading away in the distance.

- **Which output you like and why**:
    I like the pipeline output, because it's much more clear why a simile is picked.

--- 

- **Input**:
- **Baseline output**:
- **Pipeline output**:
- **Which output you like and why**:


--- 

...

## Reflect on prompting effectiveness

Write some paragraphs to describe: Did you find the pipeline prompting workflow effective? Why or Why not?

## Envision possible improvements

Write some paragraphs to describe: What are some possible ways to further improve the pipeline or prompt design?